<a href="https://colab.research.google.com/github/xinyueli2896/probing/blob/main/probing_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
# from sklearn.model_selection import train_test_split
# from transformers import AutoModel, AutoTokenizer
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import pdb

In [ ]:
num_labels = 6
num_epochs = 20

In [ ]:
!pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 1.1 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
# Load your model and tokenizer
# model_path = "/Users/lixinyue/Desktop/llama2/Llama-2-7b-hf"
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", output_hidden_states=True)
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
num_layers = model.config.num_hidden_layers
num_layers

32

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_data = pd.read_csv('/content/test.csv', header=0)
train_text = train_data['text'].values.tolist()
train_label = train_data['label'].values.tolist()
train_encodings = tokenizer(train_text, padding=True, return_tensors='pt')
train_dataset = EmotionDataset(train_encodings, train_label)

val_data = pd.read_csv('/content/training.csv', header=0)
val_text = val_data['text'].values.tolist()
val_label = val_data['label'].values.tolist()
val_encodings = tokenizer(val_text, padding=True, return_tensors='pt')
val_dataset = EmotionDataset(val_encodings, val_label)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
for i, item in enumerate(val_loader):
  print(item)
  break

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     1,   474,  5131,   304,  4459,  1075,
           297,   590,  6567,   322,  7450,   714,   304,  2125,  1075,   964,
           590, 10534, 19888, 13394],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     1,   474,  4459,   901,  9914,
           393,   982,   281,   386],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     

In [ ]:
class LinearProbe(nn.Module):
    def __init__(self, input_size=4096, num_labels=6):
        super(LinearProbe, self).__init__()
        self.fc = nn.Linear(input_size, num_labels)

    def forward(self, x):
        return self.fc(x)


class MLPProbe(nn.Module):
    def __init__(self, input_size=4096, hidden_size=1024, num_label=6):
        super(MLPProbe, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First hidden layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_labels)  # Output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [ ]:
# Training function
def train_probe(layer_idx, probe_type):
    if probe_type == 'mlp':
        probe = MLPProbe()
    elif probe_type == 'linear':
        probe = LinearProbe()
    else:
        raise ValueError(f'Unknown probe type: {probe_type}')

    optimizer = optim.AdamW(probe.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
      # Using tqdm for progress bar
      train_loop = tqdm(train_loader, total=len(train_loader), leave=False)
      b=0
      for batch in train_loop:
          probe.train()
          optimizer.zero_grad()
          input_ids = batch['input_ids']
          attention_mask = batch['attention_mask']
          labels = batch['labels']

          if layer_idx == 0 and epoch == 0:
            with torch.no_grad():
              outputs = model(input_ids, attention_mask)
            torch.save(outputs.hidden_states, f"/content/trial1/hidden_states_train_batch{b}.pt")

          hidden_states = torch.load(f"/content/trial1/hidden_states_train_batch{b}.pt")

          layer_hidden_states = hidden_states[layer_idx][:, 0, :]
          predictions = probe(layer_hidden_states)
          loss = criterion(predictions, labels)
          loss.backward()
          optimizer.step()
          train_loop.set_description(f'Epoch {epoch+1}/{num_epochs}')
          train_loop.set_postfix(loss=loss.item())
          b = b+1

    # Validate
    probe.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        test_loop = tqdm(val_loader, total=len(val_loader), leave=False)
        b=0
        for batch in test_loop:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            if layer_idx == 0:
              with torch.no_grad():
                outputs = model(input_ids, attention_mask)
              torch.save(outputs.hidden_states, f"/content/trial1/hidden_states_test_batch{b}.pt")

            hidden_states = torch.load(f"/content/trial1/hidden_states_test_batch{b}.pt")


            layer_hidden_states = outputs.hidden_states[layer_idx][:, 0, :]
            predictions = probe(layer_hidden_states)
            preds = torch.argmax(predictions, dim=1)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())
        b=b+1

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy
    # probe.eval()
    # with torch.no_grad():
    #     correct = 0
    #     total = 0
    #     for inputs, labels in val_loader:
    #         if layer_idx == 0:
    #           with torch.no_grad():
    #             outputs = model(input_ids, attention_mask)
    #           torch.save(outputs.hidden_states, f"/content/trial3/hidden_states_test_batch{b}.pt")

    #         hidden_states = torch.load(f"/content/trial3/hidden_states_test_batch{b}.pt")
    #         layer_hidden_states = hidden_states[layer_idx][:, 0, :]
    #         predictions = probe(layer_hidden_states)
    #         _, predicted = torch.max(predictions.data, 1)
    #         total += labels.size(0)
    #         correct += (predicted == labels).sum().item()

    # accuracy = correct / total
    # return accuracy



In [ ]:
# for probe in ['linear','MLP']:
# Train and evaluate the probe for each layer
num_layers = model.config.num_hidden_layers
accuracies = []
for i in range(num_layers):
    if i%5==0:
      acc = train_probe(i,'linear')
      accuracies.append(acc)
      print(f'Layer {i} for linear probe: Accuracy = {acc}')

# Analyze the accuracies to determine where the concept of emotions emerges

Epoch 16/20:  82%|████████▏ | 102/125 [08:02<01:42,  4.46s/it, loss=1.56]

In [ ]:
accuracies1 = []
for i in range(num_layers):
    if i%5==0:
      acc = train_probe(i,'mlp')
      accuracies.append(acc)
      print(f'Layer {i} for mlp probe: Accuracy = {acc}')


Layer 0 for mlp probe: Accuracy = 0.38


Layer 5 for mlp probe: Accuracy = 0.34


Layer 10 for mlp probe: Accuracy = 0.46


Layer 15 for mlp probe: Accuracy = 0.46


Layer 20 for mlp probe: Accuracy = 0.44


Layer 25 for mlp probe: Accuracy = 0.48


Layer 30 for mlp probe: Accuracy = 0.48
